In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***A guide to most of the datasets in the submission:***
* dfbar    - contains the bar dataset
* dfquote  - contains the quote dataset
* dfrating - contains the rating dataset
* dftarget - contains the target dataset
* dfevent  - contains the event dataset
* dfnews   - contains the news dataset


* dfbar5aug   - contains all the entries from bar dataset with date 5 aug 2020
* dfquote5aug - contains all the entries from quote dataset with date 5 aug 2020


* dfbarp   - contains time and average price
* dfquotep - contains time, bid price and ask price


* dfbarni    - bar dataset with no index
* dftargetni - target dataset with no index. also modified to be the solution to task 6


* dfmod  - intermediate dataset
* dfmod2 - intermediate dataset
* dfmod3 - intermediate dataset
* dfmod4 - intermediate dataset


* dftable - a table containing mean of average price in the time period from consensusStartDate to consensusEndDate
* dftab   - a table for plotting the average price of a particular stock in the time period from consensusStartDate to consensusEndDate


* dfbars  - dataset containing all the entries of a particular stock in the bar dataset
* dfbarl  - a subset of dfbars used for calculating the limit


* dftargetrd  - dataset containing all the entries on or before event date and the priceTargetAverage


* dfnewsap - the news dataset aong with average price of stocks before and after the news (and the time of the recorded average price)

# **Consider a subset ( 30 rows) of @bar. How do 'bidprice', 'askprice', 'average_price' of any stock ( any symbol) are moving in that subset period? Maybe you can try for ‘2020-08-05’. Graphical and data analysis. Dataset @quote @bar dataset**

In [ ]:
fpbar="/kaggle/input/stock-market-small-wide-dataset/bar.csv"
fpquote="/kaggle/input/stock-market-small-wide-dataset/quote.csv"

dfbar=pd.read_csv(fpbar)
dfbar.time=pd.to_datetime(dfbar.time)

dfquote=pd.read_csv(fpquote)
dfquote.time=pd.to_datetime(dfquote.time)

In [ ]:
barcolumns=list(dfbar.columns.values)
quotecolumns=list(dfquote.columns.values)
print("bar contains: ",barcolumns)
print("quote contains: ",quotecolumns)

start=pd.to_datetime('2020-08-05 00:00:00+00:00')
end=pd.to_datetime('2020-08-06 00:00:00+00:00')
dfbar5aug=dfbar[(dfbar['time']<end) & (dfbar['time']>start)]          #subset of bar dataset containing data from 5 Aug 2020
dfquote5aug=dfquote[(dfquote['time']<end) & (dfquote['time']>start)]  #subset of quote dataset containing data from 5 Aug 2020

symbol_list=dfbar5aug['symbol'].unique() #list of stocks
print(symbol_list)

In [ ]:
print("Enter the symbol: ")
sym=input() #Enter symbol in capital letters (like ABC, AAPL)

if(sym in symbol_list):
    dfbarp=dfbar5aug[dfbar5aug['symbol']==sym]
    dfquotep=dfquote5aug[dfquote5aug['ticker']==sym]
    #dfbarp=dfbarp.head(30)     #First 30 elements of stock sym on 5 aug 2020 from bar dataset
    #dfquotep=dfquotep.head(30) #First 30 elements of stock sym on 5 aug 2020 from quote dataset

    print(len(dfbarp.index))
    
    fig = plt.figure(num=sym)
    dfbarp=dfbarp[['time', 'average_price']]
    dfquotep=dfquotep[['time', 'bid_price', 'ask_price']]
    
    dfbarp.time=pd.to_datetime(dfbarp.time)
    dfbarp.set_index("time", inplace=True)
    dfquotep.time=pd.to_datetime(dfquotep.time)
    dfquotep.set_index("time", inplace=True)
    
    dfbarp.plot(kind="line", marker="o", ms=4)
    plt.title("average price")
    plt.show()
    dfquotep[['bid_price', 'ask_price']].plot(kind="line", marker="o", ms=4)
    plt.title("ask and bid price")
    plt.show()
    
else:
    print("Symbol not in the dataset")

# **make a table [symbol,ratingBuy,ratingScaleMark,mean of average_price,consensusStartDate,consensusEndDate] from @rating and @bar datasetmake a table [symbol,ratingBuy,ratingScaleMark,mean of average_price,consensusStartDate,consensusEndDate] from @rating and @bar dataset**

In [ ]:
fprating="/kaggle/input/stock-market-small-wide-dataset/rating.csv"

dfrating=pd.read_csv(fprating)
dfrating['consensusStartDate']=pd.to_datetime(dfrating['consensusStartDate'])
dfrating['consensusEndDate']=pd.to_datetime(dfrating['consensusEndDate'])
print(dfrating.info())

dftable=dfrating[["symbol","ratingBuy","ratingScaleMark","consensusStartDate","consensusEndDate"]]
dftable.reset_index(drop=True, inplace=True)
dfbarni=dfbar.reset_index(drop=True)
print(dftable.info())


for i in dftable.index:
    dfmod=dfbarni[(dfbarni['symbol']==dftable['symbol'][i]) & (dfbarni['time']<dftable['consensusEndDate'][i]) & (dfbarni['time']>dftable['consensusStartDate'][i])]
    #dfmod contains all the rows of bar dataset that have time b/w consensusStartDate and consensusEndDate and have the same symbol as the ith row of dftable
    dftable.loc[i,'meanofavg_price']=dfmod['average_price'].mean()
    #assigning the mean of average price for the time period b/w consensusStartDate and consensusEndDate to the meanofavg_price column in the dftable dataset

print(dftable.head(10))

# **'average_price' column @bar dataset. How average price changes over 'consensusStartDate' an 'consensusEndDate' from @rating dataset.Graph**

In [ ]:
print("Enter the symbol: ")
sym=input()

dftab=dfrating[dfrating['symbol']==sym]
if dftab.empty:
    print("No records of ", sym, " in rating dataset")
dftab.reset_index(drop=True, inplace=True)

for i in dftab.index:
    dfmod=dfbarni[(dfbarni['symbol']==sym) & (dfbarni['time']<dftab['consensusEndDate'][i]) & (dfbarni['time']>dftab['consensusStartDate'][i])]
    dfmod=dfmod[['time', 'average_price']]
    dfmod.set_index("time", inplace=True)
    if (dfmod.empty):
        print("Average price for ", dftab['consensusStartDate'][i], " to ", dftab['consensusEndDate'][i]," dftab not available")
        continue        
    try:
        dfmod['average_price'].plot(kind="line", marker="o", ms=2.5)
        plt.show()
    except:
        print("Average price for ", sym, " from ", dftab['consensusStartDate'][i], " to ", dftab['consensusEndDate'][i]," not available")

# **How the 'average_price' @bar dataset changes a few days before and after the 'reportDate' for any stock. Graphical.**

In [ ]:
import datetime

fpevent="/kaggle/input/stock-market-small-wide-dataset/event.csv"

dfevent=pd.read_csv(fpevent)
rDate=dfevent.iloc[2][0]
rDate=pd.to_datetime(rDate, utc=True)

print("Enter the symbol: ")
sym=input()
print("Enter the number of days: ")
nod=int(input())

days = datetime.timedelta(nod)
startDate=rDate-days
endDate=rDate+days

dfmod2=dfbar[(dfbar['symbol']==sym) & (dfbar['time']<=endDate) & (dfbar['time']>=startDate)]
dfmod2=dfmod2[['time','average_price']]
dfmod2['time']=pd.to_datetime(dfmod2['time'])

if (dfmod2.empty):
    print("No average price data for ", sym, " from ", startDate, " to ", endDate)
else:
    x_dat=dfmod2[['time']]
    y_dat=dfmod2[['average_price']]
    plt.plot(x_dat, y_dat, marker=".", ms=6)
    plt.xticks(rotation=90)
    plt.title(sym)
    plt.show()

# pull 'priceTargetAverage' column from @target dataset. Changes of 'priceTargetAverage' around 'reportDate' from @event dataset on or before. Show by table.

In [ ]:
fptarget="/kaggle/input/stock-market-small-wide-dataset/target.csv"

dftarget=pd.read_csv(fptarget)
dftargetrd=dftarget
dftargetrd['updatedDate']=pd.to_datetime(dftargetrd['updatedDate'])
dftargetrd['updatedDate']=dftargetrd['updatedDate'].dt.date
print(dftarget.info())
rd=rDate.date() 
print(type(rd))
dftargetrd=dftarget[dftarget["updatedDate"]==rd]
dftargetrd=dftargetrd[["symbol", "updatedDate", "priceTargetAverage"]]
print(dftargetrd.head())

# need to add the 'average_price' at updated date from @bar dataset

In [ ]:
dfbarni['date']=dfbarni['time']
dfbarni['date']=dfbarni['time'].dt.date
print(dfbarni.info())
dftargetni=dftarget.reset_index(drop=True)
for i in dftargetni.index:
    #dfmod3=dfbarni[(dfbarni['symbol']==dftargetni['symbol'][i]) & (dfbarni['date']==dftargetni['updatedDate'][i])] This can be used if we want only those average_price whose value has been specified on the updatedDate. So this gives more nan results in the avg_price column
    dfmod3=dfbarni[(dfbarni['symbol']==dftargetni['symbol'][i]) & (dfbarni['date']<=dftargetni['updatedDate'][i])] #This provides the closest available avg_price on or before the updatedDate
    dfmod3.sort_values("time", ascending=False, inplace=True)
    dfmod3.reset_index(drop=True)
    if (dfmod3.empty):
        dftargetni['average_price'][i]=np.nan
    else:
        k=dfmod3.head(1).index[0]
        dftargetni.loc[i, 'average_price']=dfmod3.loc[k, "average_price"]

print(dftargetni.head(10)) #targetni is the output i.e. the dataset that contains average price on (or closest to) the updated date

# need to find the trend in price prior and post updated date

In [ ]:
def plot_func(group): 
    global ax
    color = 'r' if (group['label'] < 0).all() else 'g'
    lw = 1.1
    ax.plot(group.index, group.average_price, c=color, linewidth=lw, marker='o', ms=2.5) #plotting the datapoints and asssigning color

print("Enter the symbol: ")
sym=input()
uD=[]
dfbars=dfbarni[dfbarni["symbol"]==sym]
dfbars.set_index("time", inplace=True)

if dfbars.empty:
    print(sym, "not found in the bar dataset")
else:
    for i in dftargetni.index:
         if (dftargetni['symbol'][i]==sym):
             uD.append(dftarget.loc[i, "updatedDate"])
if(len(uD)==0):
    print(sym, "not found in target dataset")
else:
    print("\nUpdated date/s for ", sym, " is/are ", uD)

    print("\nEnter the number of days prior and after the updated date for which you want to observe the trend:")
    nod=int(input())
    days = datetime.timedelta(nod)
    
    for i in range(len(uD)):
        startDate=uD[i]-days
        endDate=uD[i]+days
        
        dfbars=dfbars[(dfbars['date']>=startDate) & (dfbars['date']<=endDate)]
        if(dfbars.empty):
            print("Average price data for", sym, "from", startDate ,"to", endDate, "not found in bar dataset")
            continue
        dfbarl=dfbars[dfbars['date']==uD[i]]
        
        limit=dfbarl['average_price'].mean()  #setting the limit as mean of average_price on updatedDay
        if(np.isnan(limit)):
            limit=dfbars['average_price'].mean() #setting the limit as mean of average_price of the time period if data on the updatedDay is not available
        
        dfbars['label'] = np.where(dfbars['average_price']>limit, 1, -1)
        fig, ax = plt.subplots()
        dfbars.groupby((dfbars['label'].shift() * dfbars['label'] < 0).cumsum()).apply(plot_func)
        plt.axvline(x=uD[i], color="blue", linestyle='--', alpha=0.5)  #marks the updated Date 
        plt.plot(uD[i], limit, marker="*")  #marks the limit (listings above limit will be green and listing below will be red)
        ax.plot(dfbars.index, dfbars.average_price, color='gray',alpha=0.8, linewidth=0.7) #gray average_price lineplot to connect the various disjointed green and red trend lines
        plt.xticks(rotation=90)
        plt.title(sym)
        plt.show()

# [present trend vs.prior trend]. ranking.up/down.

In [ ]:
#present trend v/s prior trend
def plot_func(group): 
    global ax
    color = 'b' if (group['label'] < 0).all() else 'orange'
    lw = 1.1
    ax.plot(group.index, group.average_price, c=color, linewidth=lw, marker='o', ms=2.5) 

print("Enter the symbol: ")
sym=input()
uD=[]
dfbars=dfbarni[dfbarni["symbol"]==sym]
dfbars.set_index("time", inplace=True)

if dfbars.empty:
    print(sym, "not found in the bar dataset")
else:
    for i in dftargetni.index:
         if (dftargetni['symbol'][i]==sym):
             uD.append(dftarget.loc[i, "updatedDate"])
if(len(uD)==0):
    print(sym, "not found in target dataset")
else:
    print("\nUpdated date/s for ", sym, " is/are ", uD)

    print("\nEnter the number of days prior and after the updated date for which you want to observe the trend:")
    nod=int(input())
    days = datetime.timedelta(nod)
    
    for i in range(len(uD)):
        startDate=uD[i]-days
        endDate=uD[i]+days
        
        dfbars=dfbars[(dfbars['date']>=startDate) & (dfbars['date']<=endDate)]
        if(dfbars.empty):
            print("Average price data for", sym, "from", startDate ,"to", endDate, "not found in bar dataset")
            continue
        dfbarl=dfbars[dfbars['date']==uD[i]]
        
        dfbars['label'] = np.where(dfbars['date']>=uD[i], 1, -1) #check if the 
        fig, ax = plt.subplots()
        dfbars.groupby((dfbars['label'].shift() * dfbars['label'] < 0).cumsum()).apply(plot_func)
        plt.axvline(x=uD[i], color="blue", linestyle='--', alpha=0.5)  #marks the updated Date 
        ax.plot(dfbars.index, dfbars.average_price, color='gray',alpha=0.8, linewidth=0.7) #gray average_price lineplot to connect the various disjointed green and red trend lines
        plt.xticks(rotation=90)
        plt.title(sym)
        plt.show()
        print("Blue signifies prior trend. Orange signifies present trend")

In [ ]:
dfbaris=dfbarni.reset_index(drop=True)
dfbaris['date']=pd.to_datetime(dfbaris['date'])
dfbaris['date']=dfbaris['time'].dt.date
dfbaris['time']=pd.to_datetime(dfbaris['time'])
day10 = datetime.timedelta(10)
dftargett=dftarget
dftargett["score"]=np.nan
dftargett['updatedDate']=pd.to_datetime(dftargett['updatedDate'])
#dftargett['updatedDate']=dftargett['updatedDate'].dt.date
print(dftargett.info())

for i in dftargett.index:
    d1=dftargett["updatedDate"][i]+day10
    d2=dftargett["updatedDate"][i]-day10
    
    dfbaris=dfbarni[(dfbarni['symbol']==dftargett['symbol'][i])]
    dfbarpt=dfbaris[(dfbaris["date"]>=dftargett["updatedDate"][i]) & (dfbaris["date"]<=d1)]
    dfbarpr=dfbaris[(dfbaris["date"]<=dftargett["updatedDate"][i]) & (dfbaris["date"]>=d2)]
    prprice=dfbarpr['average_price'].mean()
    ptprice=dfbarpt['average_price'].mean()
    if(pd.isna(prprice) or pd.isna(ptprice)):
        sc=np.nan
    else:
        sc=(ptprice-prprice)/(prprice)  #score of stocks on every listing in the bar dataset
    
    dftargett["score"][i]=sc

dftargetrank=dftargett.sort_values("score", ascending=False)
print(dftargetrank.head()) #stocks ranked based on score

# 'datetime' upto minute ['2020-08-12 11:31'] and @averageprice at that time from @bar dataset. [datetime,stock,summary,averageprice] from @bar and @news dataset

In [ ]:
fpnews="/kaggle/input/stock-market-small-wide-dataset/news.csv"

dfnews=pd.read_csv(fpnews)

dfnews.datetime=pd.to_datetime(dfnews.datetime)
dfnews['date']=pd.to_datetime(dfnews['date'])

x="2020-08-12 11:31:00+00:00"
x=pd.to_datetime(dfnews.datetime)
dfnewsap=dfnews
dfnewsap.datetime=pd.to_datetime(dfnews.datetime)
dfnewsap['date']=pd.to_datetime(dfnewsap['date'])
dfbarni['date']=pd.to_datetime(dfbarni['date'])
day1 = datetime.timedelta(1)
day5 = datetime.timedelta(5)
print(dfnewsap.info())
print(dfbarni.info())
print(type(day1))

for i in dfnewsap.index:
    dfmod4=dfbarni[(dfbarni['symbol']==dfnewsap['stock'][i]) & (dfbarni['date']<=dfnewsap['date'][i])] #This provides the closest available avg_price on or before the date
    dfmod4.sort_values("time", ascending=False, inplace=True)
    dfmod4.reset_index(drop=True)
    if (dfmod4.empty):
        dfnewsap['last_avg_price_beforenews'][i]=np.nan
    else:
        k=dfmod4.head(1).index[0]
        dfnewsap.loc[i, 'last_avg_price_beforenews']=dfmod4.loc[k, "average_price"]
        dfnewsap.loc[i, 'time_of_avg_price_BN']=dfmod4.loc[k, "time"]
    dfmod4=dfbarni[(dfbarni['symbol']==dfnewsap['stock'][i]) & (dfbarni['date']-dfnewsap['date'][i]>=day1) & (dfbarni['date']-dfnewsap['date'][i]<=day5)] #This provides the closest available avg_price after the date
    dfmod4.sort_values("time", ascending=True, inplace=True)
    dfmod4.reset_index(drop=True)
    if (dfmod4.empty):
        dfnewsap['avg_price_afternews'][i]=np.nan
    else:
        k=dfmod4.head(1).index[0]
        dfnewsap.loc[i, 'avg_price_afternews']=dfmod4.loc[k, "average_price"]
        dfnewsap.loc[i, 'time_of_avg_price_AN']=dfmod4.loc[k, "time"]

print(dfnewsap.head(10))